In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../data/literature')
sys.path.append('..')

In [3]:
from src.experimental.train import TrainEquivariantDDPMConfig
from src.diffusion import LitEquivariantDDPM
import torch
import pandas as pd
import numpy as np
from exp_kraitchman import *
import copy
from src import chem, utils, metrics

In [50]:



# ckpt_path = '../../final_checkpoints/qm9.ckpt'
ckpt_path = '../final_checkpoints/geom.ckpt'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
ddpm = LitEquivariantDDPM.load_from_checkpoint(ckpt_path, map_location=device)
ddpm = ddpm.ema.ema_model
ddpm.eval()
ddpm.to(device)

Using device: cuda


EquivariantDDPM(
  (dynamics): TransformerDynamics(
    (embed_atom): Embedding(90, 32)
    (proj_h): Linear(in_features=171, out_features=256, bias=True)
    (proj_cond): Sequential(
      (0): Linear(in_features=171, out_features=128, bias=True)
      (1): Activation()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): Activation()
    )
    (egnn): ModuleList(
      (0): EquivariantTransformerBlock(
        (edge_mlp): Sequential(
          (0): Linear(in_features=538, out_features=320, bias=True)
          (1): Activation()
          (2): Linear(in_features=320, out_features=320, bias=True)
          (3): Activation()
        )
        (coord_mlp): Linear(in_features=320, out_features=3, bias=False)
        (val_mlp): Linear(in_features=320, out_features=256, bias=False)
        (att_mlp): Linear(in_features=320, out_features=8, bias=False)
        (out_mlp): Linear(in_features=256, out_features=256, bias=False)
        (node_mlp): Sequential(
          (0):

In [4]:
df = pd.read_csv('../data/literature/data.csv')
# df = df[~df.ground_truth.isna()]
df

,url,name,formula,rot,iso_rot,kra,ground_truth,img,in_dataset
0,https://cdnsciencepub.com/doi/10.1139/cjp-2019...,"2,3,4-trifluorotoluene",C_7 H_5 F_3,"[2085.9101, 1201.434445, 765.974221]","[['C', 2082.7023, 1195.46081, 763.1122], ['C',...","[['C', -1.4539, -0.6265, 0.0], ['C', -0.7531, ...","[['C', -1.4539, -0.6265, 0.0], ['C', -0.7531, ...",https://cdnsciencepub.com/cms/10.1139/cjp-2019...,no
1,https://cdnsciencepub.com/doi/10.1139/cjp-2019...,"2,4,5-trifluorotoluene",C_7 H_5 F_3,"[2269.3534, 1032.193, 712.627827]","[['C', 2263.7192, 1028.8396, 710.49982], ['C',...","[['C', 1.2642, 0.7302, 0.0], ['C', 1.215, -0.6...","[['C', 1.2635, 0.7493, 0.0], ['C', 1.2239, -0....",https://cdnsciencepub.com/cms/10.1139/cjp-2019...,no
2,https://pubs.rsc.org/en/content/articlelanding...,isoleucinol (I),C_6 H_15 O_1 N_1,"[3759.34376, 1062.44909, 880.24579]","[['C', 3745.439, 1055.21888, 874.74707], ['C',...",NaN,NaN,https://pubs.rsc.org/image/article/2020/cp/d0c...,no
3,https://pubs.rsc.org/en/content/articlelanding...,isoleucinol (II),C_6 H_15 O_1 N_1,"[3170.22321, 1103.61682, 971.59291]","[['C', 3153.58, 1097.5552, 965.80927], ['C', 3...",NaN,NaN,https://pubs.rsc.org/image/article/2020/cp/d0c...,no
4,https://pubs.rsc.org/en/content/articlelanding...,isoleucinol (III),C_6 H_15 O_1 N_1,"[3067.56626, 1213.42566, 1010.54858]","[['C', 3059.714, 1204.58462, 1003.61816], ['C'...",NaN,NaN,https://pubs.rsc.org/image/article/2020/cp/d0c...,no
5,https://www.sciencedirect.com/science/article/...,2-ethylfuran (C1),C_6 H_8 O_1,"[6693.72695, 1947.754374, 1656.347742]","[['O', 6523.10957, 1945.18414, 1645.14961], ['...","[['O', 0.3348, -0.9712, -0.2527], ['C', -0.143...","[['O', 0.32097, -0.97123, -0.24871], ['C', -0....",https://ars.els-cdn.com/content/image/1-s2.0-S...,yes
6,https://www.sciencedirect.com/science/article/...,2-ethylfuran (Cs),C_6 H_8 O_1,"[7104.81359, 1933.284765, 1549.965675]","[['O', 6883.1842, 1924.47193, 1533.54897], ['C...","[['O', -0.7766, -1.0839, 0.0], ['C', 0.1729, -...","[['O', -0.78108, -1.08712, -0.00015], ['C', 0....",https://ars.els-cdn.com/content/image/1-s2.0-S...,yes
7,https://pubs.acs.org/doi/full/10.1021/acscents...,2-deoxy-d-riboside (af-1),C_6 H_12 O_4,"[2056.39188, 1019.67311, 860.52781]","[['C', 2049.2702, 1016.90711, 858.93148], ['C'...","[['C', 0.891, 0.547, 0.749], ['C', 0.17, 0.628...",NaN,https://pubs.acs.org/cms/10.1021/acscentsci.9b...,no
8,https://pubs.acs.org/doi/full/10.1021/acscents...,2-deoxy-d-riboside (bf-1),C_6 H_12 O_4,"[1890.65747, 1145.05589, 910.72056]","[['C', 1882.83837, 1144.17523, 908.33842], ['C...","[['C', 0.584, 1.058, None], ['C', 0.476, 0.926...",NaN,https://pubs.acs.org/cms/10.1021/acscentsci.9b...,no
9,https://pubs.acs.org/doi/full/10.1021/acscents...,2-deoxy-d-riboside (ap-1),C_6 H_12 O_4,"[2153.8853, 1058.8982, 960.7432]","[['C', 2148.116, 1054.2212, 958.04619], ['C', ...","[['C', 1.219, None, 0.799], ['C', 0.12, 0.945,...",NaN,https://pubs.acs.org/cms/10.1021/acscentsci.9b...,no


In [5]:
# generate and rank all samples

sample_n_candidates_per_example = 100

def cond_rmse(M):
    cond_errors = torch.square(M.coords.abs() - M.cond_labels)
    cond_errors = cond_errors[M.cond_mask]
    return cond_errors.mean().sqrt().item()

all_samples = []

for i, row in df.iterrows():
    print(row['name'])
    rot = eval(row.rot)
    iso_rot = eval(row.iso_rot)

    iso_kra = kraitchman(rot, iso_rot)

    # incomplete information
    M = kra_to_molecule(None, iso_kra, rot)

    M_repeated = [copy.deepcopy(M) for _ in range(sample_n_candidates_per_example)]
    M_repeated = chem.Molecule.collate(M_repeated)
    M_repeated = M_repeated._replace(**{field: x.to(device) for field, x in M_repeated._asdict().items()})

    samples = ddpm.sample(M_repeated).cpu().unbatch()

    unsigned_rmsds = [cond_rmse(x) for x in samples]
    reorder_idxs = np.argsort(unsigned_rmsds)
    samples = [samples[i] for i in reorder_idxs]

    all_samples.append(samples)

torch.save(all_samples, '../data/literature/all_samples.pt')

2,3,4-trifluorotoluene
2,4,5-trifluorotoluene
isoleucinol (I)
isoleucinol (II)
isoleucinol (III)
2-ethylfuran (C1)
2-ethylfuran (Cs)
2-deoxy-d-riboside (af-1)
2-deoxy-d-riboside (bf-1)
2-deoxy-d-riboside (ap-1)
trans-2-naphthol
cis-2-naphthol
cis-1-naphthol
trans-1-naphthol
enflurane (G+)
N-ethylformamide
trans-cinnamaldehyde (s-trans-trans)
acetophenone
2'-aminoacetophenone
cyclopropyl (isocyanato) silane (gauche)
cyclododecanone
delta-valerolactam
verbenone
beta-estradiol (tg(+))
cycloundecanone
trans-thymol-A
linalool
strawberry aldehyde (t-aa)
strawberry aldehyde (c-sg-)
2-acetylfuran (anti)
2-methyltetrahydrofuran (equatorial)
4-hydroxy-2-butanone (I)
a-pinene oxide


In [5]:
all_samples = torch.load('../data/literature/all_samples.pt')

In [6]:
ood_idxs = df.index[(df.in_dataset == 'no') & (~df.ground_truth.isna())]

ood_M_trues = []
ood_M_preds = []
ood_sample_metrics = []
for idx in ood_idxs:
    row = df.iloc[idx]
    rot = eval(row.rot)
    iso_rot = eval(row.iso_rot)
    ground_truth = eval(row.ground_truth)

    M_GT = kra_to_molecule(ground_truth, iso_rot, rot)
    ood_M_trues.append(M_GT)
    
    generated_samples = all_samples[idx]
    M_pred = generated_samples[0]

    sample_metrics, M_aligned = metrics.evaluate_prediction(M_pred, M_GT, return_aligned_mol=True)
    ood_M_preds.append(M_aligned)
    ood_sample_metrics.append(sample_metrics)

In [7]:
from draw import add, compare_fit
import py3Dmol
n_cols = 4
n_rows = 2
width = 250
height = 250
gridview = py3Dmol.view(width=width*n_cols, height=height*n_rows, viewergrid=(n_rows, n_cols))

radius = 0.03
color = 'black'

for i in range(len(ood_M_preds)):
    M_pred = ood_M_preds[i]
    M_true = ood_M_trues[i]
    sample_metrics = ood_sample_metrics[i]

    row = i // n_cols
    col = i % n_cols

    gridview = compare_fit(gridview, M_true, M_pred, viewer=(row, col), radius=radius, color=color)
    # name = df.iloc[ood_idxs[i]]["name"]
    # gridview.addLabel(name, {"screenOffset": {"x":10, "y":0}, "useScreen": True, "showBackground": False, "fontColor": "black", "fontSize": 16, "fontFace": "Arial"}, viewer=(row, col))

    if sample_metrics['correctness']:
        bg = "#cffbcf"
    elif sample_metrics['heavy_correctness']:
        bg = "#feefb2"
    else:
        bg = "#f4ccca"
    gridview.setBackgroundColor(bg, viewer=(row, col))

gridview.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
gridview.render()
t = gridview.js()
js = t.startjs + t.endjs
with open('ood.html', 'w') as f:
    f.write(js)

In [9]:
ranks = []
for i in range(len(ood_M_preds)):
    M_true = ood_M_trues[i]

    samples = all_samples[ood_idxs[i]]
    for k in range(len(samples)):
        M_pred = samples[k]

        if metrics.connectivity_correctness(M_pred, M_true):
            break
    ranks.append(k)
ranks

[0, 0, 0, 99, 1, 0, 0, 0]

In [10]:
in_idxs = df.index[(df.in_dataset == 'yes') & (~df.ground_truth.isna())]

in_M_trues = []
in_M_preds = []
in_sample_metrics = []
for idx in in_idxs:
    row = df.iloc[idx]
    rot = eval(row.rot)
    iso_rot = eval(row.iso_rot)
    ground_truth = eval(row.ground_truth)

    M_GT = kra_to_molecule(ground_truth, iso_rot, rot)
    in_M_trues.append(M_GT)
    
    generated_samples = all_samples[idx]
    M_pred = generated_samples[0]

    sample_metrics, M_aligned = metrics.evaluate_prediction(M_pred, M_GT, return_aligned_mol=True)
    in_M_preds.append(M_aligned)
    in_sample_metrics.append(sample_metrics)

In [11]:
from draw import add, compare_fit
import py3Dmol
n_cols = 4
n_rows = 2
width = 250
height = 250
gridview = py3Dmol.view(width=width*n_cols, height=height*n_rows, viewergrid=(n_rows, n_cols))

linelength = 20
radius = 0.03
color = 'black'

for i in range(len(in_M_preds)):
    M_pred = in_M_preds[i]
    M_true = in_M_trues[i]
    sample_metrics = in_sample_metrics[i]

    row = i // n_cols
    col = i % n_cols

    gridview = compare_fit(gridview, M_true, M_pred, viewer=(row, col), radius=radius, color=color)
    # name = df.iloc[in_idxs[i]]["name"]
    # gridview.addLabel(name, {"screenOffset": {"x":10, "y":0}, "useScreen": True, "showBackground": False, "fontColor": "black", "fontSize": 16, "fontFace": "Arial"}, viewer=(row, col))


    if sample_metrics['correctness']:
        bg = "#cffbcf"
    elif sample_metrics['heavy_correctness']:
        bg = "#feefb2"
    else:
        bg = "#f4ccca"
    gridview.setBackgroundColor(bg, viewer=(row, col))

gridview.zoom(.7, viewer=(1, 0))
gridview.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [12]:
gridview.render()
t = gridview.js()
js = t.startjs + t.endjs
with open('in.html', 'w') as f:
    f.write(js)

In [13]:
ranks = []
for i in range(len(in_M_trues)):
    M_true = in_M_trues[i]

    samples = all_samples[in_idxs[i]]
    for k in range(len(samples)):
        M_pred = samples[k]

        if metrics.connectivity_correctness(M_pred, M_true):
            break
    ranks.append(k)
ranks

[0, 0, 0, 0, 51, 0, 0]

In [14]:
no_gt_idxs = df.index[df.ground_truth.isna()]
no_gt_M_preds = [all_samples[idx][0] for idx in no_gt_idxs]

In [15]:
for idx in no_gt_idxs:
    row = df.iloc[idx]
    img = row.img
    name = row['name']
    print(name, img, row.url)

isoleucinol (I) https://pubs.rsc.org/image/article/2020/cp/d0cp01141j/d0cp01141j-f2_hi-res.gif https://pubs.rsc.org/en/content/articlelanding/2020/CP/d0cp01141j
isoleucinol (II) https://pubs.rsc.org/image/article/2020/cp/d0cp01141j/d0cp01141j-f2_hi-res.gif https://pubs.rsc.org/en/content/articlelanding/2020/CP/d0cp01141j
isoleucinol (III) https://pubs.rsc.org/image/article/2020/cp/d0cp01141j/d0cp01141j-f2_hi-res.gif https://pubs.rsc.org/en/content/articlelanding/2020/CP/d0cp01141j
2-deoxy-d-riboside (af-1) https://pubs.acs.org/cms/10.1021/acscentsci.9b01277/asset/images/large/oc9b01277_0002.jpeg https://pubs.acs.org/doi/full/10.1021/acscentsci.9b01277
2-deoxy-d-riboside (bf-1) https://pubs.acs.org/cms/10.1021/acscentsci.9b01277/asset/images/large/oc9b01277_0002.jpeg https://pubs.acs.org/doi/full/10.1021/acscentsci.9b01277
2-deoxy-d-riboside (ap-1) https://pubs.acs.org/cms/10.1021/acscentsci.9b01277/asset/images/large/oc9b01277_0002.jpeg https://pubs.acs.org/doi/full/10.1021/acscentsci.

In [16]:
correctnesses = [
    'yes', 'no', 'yes', 'yes', 'yes', 'yes',
    'maybe', 'yes', 'yes', 'yes', 'yes', 'yes',
    'yes', 'yes', 'no', 'yes', 'no', 'maybe',
]

In [17]:
# isoleucinol (II) found in top 4
# trans-2-naphthol not found in top 10
# trans-thymol-A not found in top 10
# strawberry aldehyde (c-sg-) found in top 4
# 4-hydroxy-2-butanone (I) found in top 4

In [18]:
# get indices of 'no' in correctnesses
no_idxs = [i for i, x in enumerate(correctnesses) if x != 'yes']

n_cols = 10
n_rows = 5
width = 250
height = 250
gridview = py3Dmol.view(width=width*n_cols, height=height*n_rows, viewergrid=(n_rows, n_cols))

radius = 0.03
color = 'black'

for i in range(len(no_idxs)):
    for j in range(10):
        M_pred = all_samples[no_gt_idxs[no_idxs[i]]][j]

        viewer = (i, j)
        gridview.addModel(M_pred.replace(coords=M_pred.coords*torch.tensor([-1, 1, 1])).xyzfile(), "xyz", viewer=viewer)
        gridview.setStyle({'model': -1}, {'sphere':{'scale': 0.5}}, viewer=viewer)
gridview.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [19]:
from draw import add, compare_fit
import py3Dmol
n_cols = 6
n_rows = 3
width = 250
height = 250
gridview = py3Dmol.view(width=width*n_cols, height=height*n_rows, viewergrid=(n_rows, n_cols))

radius = 0.03
color = 'black'

for i in range(len(no_gt_M_preds)):
    M_pred = no_gt_M_preds[i]

    row = i // n_cols
    col = i % n_cols

    viewer = (row, col)
    # gridview.addModel(M_pred.replace(coords=M_pred.coords*torch.tensor([-1, 1, 1])).xyzfile(), "xyz", viewer=viewer)
    gridview.addModel(M_pred.xyzfile(), "xyz", viewer=viewer)
    gridview.setStyle({'model': -1}, {'sphere':{'scale': 0.5}}, viewer=viewer)

    if correctnesses[i] == 'yes':
        bg = "#cffbcf"
    elif correctnesses[i] == 'maybe':
        bg = "#feefb2"
    else:
        bg = "#f4ccca"
    gridview.setBackgroundColor(bg, viewer=(row, col))

gridview.zoom(.8, viewer=(2, 3))
gridview.zoom(.9, viewer=(1, 5))
gridview.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [20]:
gridview.render()
t = gridview.js()
js = t.startjs + t.endjs
with open('no_gt.html', 'w') as f:
    f.write(js)